### Vistra 10 y data plots

In [343]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import math
import seaborn as sns
import matplotlib.patches as mpatches

#importing libraries for building widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

# Suppress all warnings
warnings.filterwarnings('ignore')

#importing somemore libraries for building widgets
import panel as pn
from panel.interact import interact
pn.extension()
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

### Connecting directly to snowflake

In [344]:
pip install snowflake-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [345]:
from snowflake.connector import connect
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
from calendar import monthrange


# read data from snowflake
conn = connect(
        user='BALAPP',
        password='HTflake1',
        account='uqa28410.us-east-1',
        database='YES',
        schema='YESDATA',
        warehouse='SMALL')

def read_sf(sql, conn=conn):
    cursor = conn.cursor()
    cursor.execute(sql)
    df = cursor.fetch_pandas_all()
    cursor.close()
    return df

### Reading both CEMS and LPI data

In [346]:
#reading 10 year vistra generation data from snowflake
vistra_10y = read_sf("""select
SUM(gen.value) AS gross_load,
pl.plant_code AS plant_code,
pl.plant_name AS plant_name,
pl.state as state,
TO_CHAR (datetime, 'YYYY-MM-DD HH') as date
from TS_CEMS_PLANT_GEN_V gen
left join DS_PLANTS pl
    ON pl.objectid = gen.objectid
where
    TO_CHAR (datetime, 'YYYY-MM-DD HH') >= '2014-01-01 00'
    and plant_code in (
'6146', '55480', '6180', '55091', '55736', '55131', '54547', '889', '876', '55097', '55215', '55144', '260', '3452', '55149', '55320', '55516', '55397', '3504', '6178', '3490', '6017', '55231', '55193', '55126', '55211', '55212', '55068', '2880', '3492', '55349', '55296', '10633', '55223', '10308', '3494', '10726', '8063', '3507', '2831'
)
GROUP BY
TO_CHAR (datetime, 'YYYY-MM-DD HH'),
pl.plant_code,
pl.plant_name,
pl.state""")

In [347]:
#reading 10 year vistra lpi generation data from snowflake
vistra_10y_lpi = read_sf("""select
SUM(gen.value)/12 AS gross_load,
pl.plant_code AS plant_code,
pl.plant_name AS plant_name,
pl.state as state,
TO_CHAR (datetime, 'YYYY-MM-DD HH') as date
from LPI_GEN gen
left join object_relationships o1
    ON o1.objectid1 = gen.objectid
left join DS_PLANTS pl
    ON o1.objectid2 = pl.objectid
where
    TO_CHAR (datetime, 'YYYY-MM-DD HH') >= '2014-01-01 00'
    and plant_code in (
'6146', '55480', '6180', '55091', '55736', '55131', '54547', '889', '876', '55097', '55215', '55144', '260', '3452', '55149', '55320', '55516', '55397', '3504', '6178', '3490', '6017', '55231', '55193', '55126', '55211', '55212', '55068', '2880', '3492', '55349', '55296', '10633', '55223', '10308', '3494', '10726', '8063', '3507', '2831'
)
GROUP BY
TO_CHAR (datetime, 'YYYY-MM-DD HH'),
pl.plant_code,
pl.plant_name,
pl.state""")

In [348]:
vistra_10y_lpi['GROSS_LOAD'] = vistra_10y_lpi['GROSS_LOAD'].astype(float)
vistra_10y['GROSS_LOAD'] = vistra_10y['GROSS_LOAD'].astype(float)

In [349]:
# Reset display options to defaults
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [350]:
#reading vistra key data (plant ids and their details)
vistrakey = pd.read_csv("vistra_key.csv")

In [351]:
#cleaning data for merging
vistrakey.columns = vistrakey.columns.str.lower()
vistra_10y.columns = vistra_10y.columns.str.lower()
vistra_10y_lpi.columns = vistra_10y_lpi.columns.str.lower()
vistra_10y.rename(columns={"plant_code":"plant id"}, inplace=True)
vistra_10y_lpi.rename(columns={"plant_code":"plant id"}, inplace=True)

### Clubbing both CEMS and LPI data

In [352]:
#handling datetime issues
vistra_10y[['year', 'month', 'day-hour']] = vistra_10y['date'].str.split('-', expand=True)

vistra_10y[['day','hour']] = vistra_10y['day-hour'].str.split(' ', expand=True)

# Pad month and day with leading zeros to get 01,02 etc
vistra_10y['month'] = vistra_10y['month'].str.zfill(2)
vistra_10y['day'] = vistra_10y['day'].str.zfill(2)
vistra_10y['hour'] = vistra_10y['hour'].str.zfill(2)

# Combine the padded month, day, and year into the desired format
vistra_10y['yearmo'] =  vistra_10y['year'] + vistra_10y['month']

#handling datetime issues
vistra_10y_lpi[['year', 'month', 'day-hour']] = vistra_10y_lpi['date'].str.split('-', expand=True)

vistra_10y_lpi[['day','hour']] = vistra_10y['day-hour'].str.split(' ', expand=True)

# Pad month and day with leading zeros to get 01,02 etc
vistra_10y_lpi['month'] = vistra_10y_lpi['month'].str.zfill(2)
vistra_10y_lpi['day'] = vistra_10y_lpi['day'].str.zfill(2)
vistra_10y_lpi['hour'] = vistra_10y_lpi['hour'].str.zfill(2)

# Combine the padded month, day, and year into the desired format
vistra_10y_lpi['yearmo'] =  vistra_10y_lpi['year'] + vistra_10y_lpi['month']

In [353]:
vistra_10y.drop(labels=['day-hour'], axis = 'columns', inplace = True)
vistra_10y_lpi.drop(labels=['day-hour'], axis = 'columns', inplace = True)

In [354]:
#removing the yearmos with incomplete data (less than 30% of max data)
valid_yearmos = vistra_10y.groupby(['yearmo'])['date'].count()[~(vistra_10y.groupby(['yearmo'])['date'].count() < 0.3*(vistra_10y.groupby(['yearmo'])['date'].count().max()))].index

In [355]:
vistra_10y = vistra_10y[vistra_10y['yearmo'].isin(valid_yearmos)]

In [356]:
vistra_10y_lpi = vistra_10y_lpi[(vistra_10y_lpi['yearmo'] > vistra_10y['yearmo'].max()) & (vistra_10y_lpi['yearmo'] < vistra_10y_lpi['yearmo'].max())]

In [357]:
vistra_10y = pd.concat([vistra_10y,vistra_10y_lpi], axis = 'rows', ignore_index = True)

In [358]:
vistrakey.drop(labels= ['unnamed: 7','unnamed: 8'], axis = 'columns', inplace = True)

### Merging the data with vistra key

In [359]:
#merging both the datasets
merged_data = vistra_10y.merge(vistrakey, on="plant id", how = "left")

In [360]:
# creating a newcolum that has quarter
merged_data['quarter'] = np.where(merged_data['month'].isin(['01', '02', '03']), 'Quarter 1',
                                  np.where(merged_data['month'].isin(['04', '05', '06']), 'Quarter 2',
                                           np.where(merged_data['month'].isin(['07', '08', '09']), 'Quarter 3',
                                                    'Quarter 4')))

Month day relationships for denominator of capacity factor

In [361]:
def get_month_days(yearmo):
    # Check if the year is a leap year
    year = int(yearmo[:4])
    month = yearmo[4:]
    is_leap_year = (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)

    # Create the month_days dictionary
    month_days = {
        '01': 31,  # January
        '02': 29 if is_leap_year else 28,  # February
        '03': 31,  # March
        '04': 30,  # April
        '05': 31,  # May
        '06': 30,  # June
        '07': 31,  # July
        '08': 31,  # August
        '09': 30,  # September
        '10': 31,  # October
        '11': 30,  # November
        '12': 31   # December
    }


    return month_days[month]

### Calculating capcity and coverage

In [362]:
#rated generation: the maximum generation of a powerplant
merged_data['rated_generation_for_hour'] = merged_data['capacity(mw)']

In [363]:
#number of days the plant was operational
coverage_in_month = merged_data.groupby(['plant id','yearmo'])['date'].count().reset_index()

### Bringing data to month x plant x hour level

In [364]:
merged_data_month_hour = merged_data.groupby(['yearmo','quarter','year','month','hour','plant id','plant_name','state','plant','segment','ba','zone','type','capacity(mw)'])[['gross_load']].sum().reset_index()

In [365]:
#rated generation: the maximum generation of a powerplant
merged_data_month_hour['month_hour_rated_generation'] = merged_data_month_hour['yearmo'].apply(get_month_days)*merged_data_month_hour['capacity(mw)']

In [366]:
merged_data_month_hour = merged_data_month_hour.merge(coverage_in_month, on=['plant id','yearmo'], how='left')

In [367]:
merged_data_month_hour.rename(columns = {'date':'coverage'}, inplace=True)

### Identifying the important ISO

In [368]:
merged_data_month_hour['ba'] = np.where(
    merged_data_month_hour['ba'].isin(['ERCOT', 'PJM', 'ISO-NE']),
    merged_data_month_hour['ba'],'NYISO,MISO,CAISO'
)
merged_data_month_hour['all_ba'] = 'All BAs'

In [369]:
current_year = int(merged_data_month_hour['year'].max())

In [370]:
regions = (list(merged_data_month_hour['ba'].unique()) + list(merged_data_month_hour['all_ba'].unique()))
regions.reverse()

years = [str(current_year - x) for x in [0, 1, 2, 3, 4, 5]]

### Calculating the capacity factor by plant

In [371]:
merged_data_month_hour['cf'] = (merged_data_month_hour['gross_load']/merged_data_month_hour['month_hour_rated_generation'] * 100).round(2)

In [372]:
plants = merged_data_month_hour['plant_name'].unique()
plants.sort()
hours = merged_data_month_hour['hour'].unique()
hours.sort()

### Plotting plant x hour level

In [373]:
from ipywidgets import interact, fixed

def plot_capacity_factor_for_one_plant(data, plant, hour):

    plot_df = data[data['plant_name'] == plant]
    plot_df = plot_df[plot_df['hour'] == hour]

    if plot_df.empty:
        print(f"No data available for {plant}.")
        return

    plot_df['month'] = plot_df['month'].astype(int)
    fig, ax = plt.subplots(figsize=(8, 5))

    years_interest = [str(current_year - x) for x in [0, 1, 2]]
    years_average = [str(current_year - x) for x in [1, 2, 3, 4, 5]]

    for year in years_interest:
        temp_df = plot_df[plot_df['year'] == year]
        temp_df['cf'] = (temp_df['gross_load'] / temp_df['month_hour_rated_generation']  * 100).round(2)
        sns.lineplot(data=temp_df, x='month', y='cf', label=year, linewidth=2.5, ax=ax, marker = 'o')

    temp_df_average = plot_df[plot_df['year'].isin(years_average)]
    avg_data = temp_df_average.groupby(['month'])['cf'].mean().reset_index()
    sns.lineplot(data=avg_data, x='month', y='cf', label='Previous 5 Year Mean', linestyle='--', linewidth=2.5, ax=ax, marker = 'o')

    # Calculate the minimum and maximum bounds for the shaded area
    lower_bound = temp_df_average.groupby(['month'])['cf'].min().reset_index()['cf']
    upper_bound = temp_df_average.groupby(['month'])['cf'].max().reset_index()['cf']
    months = temp_df_average['month'].sort_values().unique()

    # Fill between the minimum and maximum values
    ax.fill_between(months, lower_bound, upper_bound, color='lightblue', alpha=0.3)

    ax.set_xticks(range(1, 13))
    ax.set_xticklabels([str(i) for i in range(1, 13)])
    ax.set_xlabel('Month', fontsize=10)
    ax.set_ylabel('Capacity Factor (%)', fontsize=10)
    ax.set_title(plant, fontsize=12)

    # Create a custom legend with the shaded area
    range_patch = mpatches.Patch(color='lightblue', alpha=0.3, label='5-Year Min/Max Range')
    handles, labels = ax.get_legend_handles_labels()
    handles.append(range_patch)  # add the custom patch to existing handles
    ax.legend(handles=handles, loc='upper right', fontsize=7)

    plt.tight_layout()
    plt.show()

# Example usage, ensure 'cf_data_by_ba' and 'regions' are defined
interact(plot_capacity_factor_for_one_plant, data=fixed(merged_data_month_hour), plant=plants, hour=hours)

interactive(children=(Dropdown(description='plant', options=('ANP Bellingham Energy Project', 'ANP Blackstone …

<function __main__.plot_capacity_factor_for_one_plant(data, plant, hour)>

In [375]:
merged_data['datekey'] = merged_data['year'] + merged_data['month'] + merged_data['day']

In [392]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def plot_capacity_factor_for_one_plant(data, plant, hours):

    plot_df = data[data['plant_name'] == plant]
    plot_df = plot_df[plot_df['year'] == '2023'] # year of interest

    if plot_df.empty:
        print(f"No data available for {plant}.")
        return

    plot_df['month'] = plot_df['month'].astype(int)
    plot_df['datekey'] = pd.to_datetime(plot_df['datekey'], format='%Y%m%d')
    fig, ax = plt.subplots(figsize=(10, 5))

    base_color = sns.color_palette("Blues", len(hours))

    for i, timeperiod in enumerate(hours):
        temp_df = plot_df[plot_df['hour'] == timeperiod][['datekey', 'rated_generation_for_hour', 'gross_load']]
        temp_df['cf'] = (temp_df['gross_load'] / temp_df['rated_generation_for_hour'] * 100).round(2)
        sns.lineplot(data=temp_df, x='datekey', y='cf', label=timeperiod, linewidth=2.5, ax=ax, color=base_color[i])

    ax.set_xlabel('Date', fontsize=10)
    ax.set_ylabel('Capacity Factor (%)', fontsize=10)
    ax.set_title(plant, fontsize=12)

    # Set datekey as index for resampling
    temp_df.set_index('datekey', inplace=True)
    
    # Setting x-ticks to the first day of each month
    monthly_ticks = temp_df.resample('MS').first().index
    ax.set_xticks(monthly_ticks)
    ax.set_xticklabels(monthly_ticks.strftime('%b %d'), rotation=45, ha='right')

    ax.legend(loc='upper right', fontsize=7)

    plt.tight_layout()
    plt.show()

# Example usage
#plot_capacity_factor_for_one_plant(merged_data, "Martin Lake", hours)
interact(plot_capacity_factor_for_one_plant, data=fixed(merged_data), plant=plants, hours=fixed(hours))

interactive(children=(Dropdown(description='plant', options=('ANP Bellingham Energy Project', 'ANP Blackstone …

<function __main__.plot_capacity_factor_for_one_plant(data, plant, hours)>

### Plotting all three on the same chart

In [ ]:
def plot_plants_by_year(plant, data, hour):

  plot_df = data
  plot_df = plot_df[plot_df['plant_name'] == plant]
  plot_df = plot_df[plot_df['hour'] == hour] 

  #aggregating at an annual level
  plot_df_annual = plot_df.groupby(['year'])[['month_hour_rated_generation','gross_load']].sum().reset_index()

  #calculating cf at an annual level
  plot_df_annual['cf'] = (plot_df_annual['gross_load']/plot_df_annual['month_hour_rated_generation']*100).round(2).astype(int)

  if plot_df_annual.empty:
    print(f"No data available for {plant} at hour {hour}.")
    return

  # Create the plot
  plt.figure(figsize=(8, 4))
  sns.set(style="whitegrid")
  sns.lineplot(data=plot_df_annual, x='year', y='cf', marker='o')

  # Customize the plot
  plt.title('Annual Capacity Factor of ' + plant, fontsize=10)
  plt.xlabel('Year', fontsize=10)
  plt.ylabel('Capacity Factor (%)', fontsize=10)
  plt.xticks(rotation=45, fontsize = 8)
  plt.tight_layout()

  # Show the plot
  plt.show()

interact(plot_plants_by_year, plant = plants, data= fixed(merged_data_month_hour), hour = hours)

### Diagnostic Table

In [ ]:
years = ['2024','2023','2022','2021','2020']
views = ['month','quarter']
months = ['January','February','March','Quarter 1','April','May']#,'June','Quarter 2','July','August','September','Quarter 3','October','November','December','Quarter 4']

In [ ]:
# region = pn.widgets.Select(name='Region', options=regions)
# month = pn.widgets.Select(name='Month', options=months)
# #data = fixed(merged_data_month_plant_level)  # Assuming merged_data_month_plant_level is defined elsewhere

# data = merged_data_month_plant_level

# @pn.depends(region=region.param.value, month=month.param.value)
def tabular_view(plant, data, hour, view, year):
    current_year = int(year)
    merged_data_month_hour = data

    # converting the data to a quarterly level if quarter time frame is requested
    item = view
        
    # for handling if quarter is requested
    merged_data_month_hour = merged_data_month_hour.groupby([item,'hour','year','plant id','plant_name','state','plant','segment','ba','zone','type','all_ba']).agg({
          'capacity(mw)': 'mean',
          'gross_load': 'sum',
          'month_hour_rated_generation':'sum',
          'coverage':'mean'
      }).reset_index()

    # filtering out for the requested time period
    merged_data_month_hour = merged_data_month_hour[merged_data_month_hour['hour'] == hour]
    merged_data_month_hour = merged_data_month_hour[merged_data_month_hour['plant_name'] == plant]

    #creating a dataframe to concatenate data of all plants
    plant_level_views_tab_all_months = pd.DataFrame(columns = [item, 'hour','year','plant id', 'plant_name', 'state', 'plant', 'segment',
          'ba', 'zone','type', 'all_ba','capacity(mw)','gross_load','month_hour_rated_generation' ,'coverage', 'cf', 'difference_in_generation','difference_in_generation(%)','previous_year_gross_load'])

    for timeperiod in merged_data_month_hour[item].unique():
        #filtering out for a single plant and the requested timeframe
        month_level_views = merged_data_month_hour[merged_data_month_hour[item] == timeperiod]
        month_level_views1 = month_level_views[month_level_views['year'] == str(current_year)]

        #getting last years data during the same month
        previous_year = [str(current_year - x) for x in [1]]
        month_level_views2 = month_level_views[month_level_views['year'].isin(previous_year)]
        month_level_views2.rename(columns = {'gross_load':'previous_year_gross_load'}, inplace=True)
        month_level_views1 = month_level_views1[[item,'hour', 'year','plant id', 'plant_name', 'state', 'plant', 'segment','ba', 'zone','type',
                                                'all_ba','capacity(mw)','gross_load','month_hour_rated_generation' ,'coverage']]

        #it is important to maintain index when concating along columns
        month_level_views1 = month_level_views1.reset_index(drop=True)
        month_level_views2 = month_level_views2.reset_index(drop=True)
        month_level_views_tab = pd.concat([month_level_views1,month_level_views2.loc[0:len(month_level_views1)-1,'previous_year_gross_load']], axis = 'columns')
        
        #calculating cf and other quantities at a plant level
        month_level_views_tab['cf'] = (month_level_views_tab['gross_load'] / month_level_views_tab['month_hour_rated_generation'])
        month_level_views_tab['difference_in_generation'] = (month_level_views_tab['gross_load'] - month_level_views_tab['previous_year_gross_load'])
        month_level_views_tab['difference_in_generation(%)'] = (month_level_views_tab['gross_load'] - month_level_views_tab['previous_year_gross_load'])/month_level_views_tab['gross_load']*100

        #concatenating all plant level data to the main dataframe
        plant_level_views_tab_all_months = pd.concat([plant_level_views_tab_all_months,month_level_views_tab], axis = 'rows')

    
    plant_level_views_tab_all_months = plant_level_views_tab_all_months.sort_values(by = [item],ascending = [True])
    
    plant_level_views_tab_all_months.rename(columns={'plant_name':'plant name','gross_load':'current year generation(mwh)','difference_in_generation':'difference in generation(mwh)','difference_in_generation(%)':'difference in generation(%)','cf':'capacity factor(%)'},inplace=True)
    plant_level_views_tab_all_months = plant_level_views_tab_all_months[[item,'year','plant id', 'plant name', 'state','segment','ba','type','capacity(mw)','current year generation(mwh)','difference in generation(mwh)','difference in generation(%)','capacity factor(%)']]

    plant_level_views_tab_all_months['capacity(mw)'] = (plant_level_views_tab_all_months['capacity(mw)']).round(0).astype(int)
    plant_level_views_tab_all_months['current year generation(mwh)'] = (plant_level_views_tab_all_months['current year generation(mwh)']).round(0).astype(int)
    plant_level_views_tab_all_months['difference in generation(mwh)'] = (plant_level_views_tab_all_months['difference in generation(mwh)']).round(0).astype(int)
    plant_level_views_tab_all_months['difference in generation(%)'] = (plant_level_views_tab_all_months['difference in generation(%)']).round(0).astype(int)
    plant_level_views_tab_all_months['capacity factor(%)'] = (plant_level_views_tab_all_months['capacity factor(%)']*100).round(0).astype(int)

    display(plant_level_views_tab_all_months.reset_index(drop=True))
interact(tabular_view, plant = plants, data= fixed(merged_data_month_hour), hour = hours, view = views, year = years)